# Multi-Gaussian Cross-Covariance Tests

Tests for cross-covariance handling between mflike and lensing.

**Note:** Run `01_multi_gaussian_setup.ipynb` first, or copy the setup cells from it.

In [ ]:
%matplotlib inline
# import tempfile

import os

import camb
import cobaya
import matplotlib as mpl
import numpy as np
from cobaya.tools import resolve_packages_path

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

In [ ]:
# Fiducial cosmo params. When used in a MCMC, the smooth data must agree
# with this input values

cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": 0.06,
    "nnu": 3.044,
}

common_nuis_params = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0,  # only ideal values for now
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

TT_nuis_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "xi": 0.20,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.0,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
}

TE_nuis_params = {
    "a_gte": 0.10,
    "a_pste": 0,
    "alpha_dE": -0.4,
}

EE_nuis_params = {
    "a_gee": 0.10,
    "a_psee": 0,
    "alpha_dE": -0.4,
    "calE_LAT_93": 1,
    "calE_LAT_145": 1,
    "calE_LAT_225": 1,
}

In [ ]:
custom_packages_path = None

# Try to resolve global path
if custom_packages_path is None:
    packages_path = resolve_packages_path()
else:
    packages_path = custom_packages_path

In [ ]:
namedir = packages_path + "/data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
mflike_options = {
    "data_folder": namedir,
    "input_file": pre + "smooth_00000.fits",
    "cov_Bbl_file": packages_path + "/data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
    "defaults": {
        "polarizations": ["TT", "TE", "ET", "EE"],
        "scales": {
            "TT": [30, 9000],
            "TE": [30, 9000],
            "ET": [30, 9000],
            "EE": [30, 9000],
        },
        "symmetrize": False,
    },
}

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
    "theory_lmax": 5000,
    "stop_at_error": True,
}

info_multi = {
    "likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {
        "camb": {
            "extra_args": {
                "bbn_predictor": "PArthENoPE_880.2_standard.dat",
                "WantTransfer": True,
                "Transfer.high_precision": True,
                "Transfer.kmax": 1.2,
                "halofit_version": "mead",
                "lens_potential_accuracy": 1,
                "num_massive_neutrinos": 1,
                "theta_H0_range": [20, 100],
            },
            "stop_at_error": True,
        }
    },
    "params": cosmo_params
    | common_nuis_params
    | TT_nuis_params
    | TE_nuis_params
    | EE_nuis_params,
    "packages_path": packages_path,
    "debug": False,
}
info_multi["theory"]["mflike.BandpowerForeground"] = {"stop_at_error": True}

In [ ]:
from cobaya.model import get_model

model_multi = get_model(info_multi)

In [ ]:
model_multi.loglikes()

In [ ]:
model_multi.components

In [ ]:
Dl = model_multi.components[2].get_Cl(ell_factor=True)
fg_totals = model_multi.components[3].get_fg_totals()

In [ ]:
params = {
    **cosmo_params,
    **common_nuis_params,
    **TT_nuis_params,
    **EE_nuis_params,
    **TE_nuis_params,
}

In [ ]:
mflike = model_multi.components[0].likelihoods[0]
dls = {s: Dl[s][mflike.l_bpws] for s, _ in mflike.lcuts.items()}
# combine CMB and FG and add svstematics
DlsObs = mflike.get_modified_theory(dls, fg_totals, **params)

In [ ]:
ps_dic = {}
ps_vec = np.zeros_like(mflike.data_vec)

for m in mflike.spec_meta:
    p = m["pol"]
    ids = m["ids"]
    w = m["bpw"]
    t1 = m["t1"]
    t2 = m["t2"]

    # print(p, ids, t1, t2)
    if t1 + "x" + t2 not in ps_dic.keys():
        ps_dic[t1 + "x" + t2] = {"lbin": m["leff"]}

    dls_obs = (
        DlsObs[p, m["t2"], m["t1"]] if m["hasYX_xsp"] else DlsObs[p, m["t1"], m["t2"]]
    )
    for i, nonzero, weights in zip(m["ids"], w.nonzeros, w.sliced_weights):
        ps_vec[i] = weights @ dls_obs[nonzero]

    ps_dic[t1 + "x" + t2].update({p: ps_vec[ids]})

In [ ]:
ps_dic.keys()  # check that all spectra are here

In [ ]:
namedir = packages_path + "/data/soliket_mflike/smooth_data/split_mflike/"
os.makedirs(namedir, exist_ok=True)

for k in ps_dic.keys():
    namefile = "Dl_frommulti_" + k + "_auto_00000.dat"
    ll = ps_dic[k]["lbin"]
    tt = ps_dic[k]["tt"]
    te = ps_dic[k]["te"]
    ee = ps_dic[k]["ee"]
    tbebbb = np.zeros(len(ll))
    np.savetxt(
        namedir + namefile,
        np.column_stack((ll, tt, te, tbebbb, te, tbebbb, ee, tbebbb, tbebbb, tbebbb)),
    )

In [ ]:
cltest = np.loadtxt(namedir + namefile)
cltest.shape

In [ ]:
data = {}
sim_suffix = "00000"
for spec_name in ps_dic.keys():
    na, nb = spec_name.split("x")
    data[na, nb] = {}
    spec = np.loadtxt(
        "%s/Dl_frommulti_%s_auto_%s.dat" % (namedir, spec_name, sim_suffix), unpack=True
    )
    ps = {
        "lbin": spec[0],
        "TT": spec[1],
        "TE": spec[2],
        "TB": spec[3],
        "ET": spec[4],
        "BT": spec[5],
        "EE": spec[6],
        "EB": spec[7],
        "BE": spec[8],
        "BB": spec[9],
    }
    data[na, nb] = ps

In [ ]:
exp_freq = ["LAT_93", "LAT_145", "LAT_225"]
pols = ["T", "E", "B"]
map_types = {"T": "0", "E": "e", "B": "b"}


def get_x_iterator():
    for id_efa, efa in enumerate(exp_freq):
        for id_efb, efb in enumerate(exp_freq):
            if id_efa > id_efb:
                continue
            for ipa, pa in enumerate(pols):
                if efa == efb:
                    polsb = pols[ipa:]
                else:
                    polsb = pols
                for pb in polsb:
                    yield (efa, efb, pa, pb)
                    print(efa, efb, pa, pb)

In [ ]:
import sacc

spec_sacc = sacc.Sacc()

In [ ]:
for exp_f in exp_freq:
    print("%s_s0" % (exp_f))

    my_data_bandpasses = {
        "nu": np.array([float(exp_f.split("_")[1])]),
        "b_nu": np.array([1.0]),
    }
    my_data_beams = {"l": np.arange(10000), "bl": np.ones(10000)}

    # CMB temperature
    spec_sacc.add_tracer(
        "NuMap",
        "%s_s0" % (exp_f),
        quantity="cmb_temperature",
        spin=0,
        nu=mflike.bands[exp_f + "_s0"]["nu"],
        bandpass=mflike.bands[exp_f + "_s0"]["bandpass"],
        ell=my_data_beams["l"],
        beam=my_data_beams["bl"],
    )

    # CMB polarization
    spec_sacc.add_tracer(
        "NuMap",
        "%s_s2" % (exp_f),
        quantity="cmb_polarization",
        spin=2,
        nu=mflike.bands[exp_f + "_s2"]["nu"],
        bandpass=mflike.bands[exp_f + "_s2"]["bandpass"],
        ell=my_data_beams["l"],
        beam=my_data_beams["bl"],
    )

In [ ]:
for id_x, (efa, efb, pa, pb) in enumerate(get_x_iterator()):
    if pa == "T":
        ta_name = "%s_s0" % (efa)
    else:
        ta_name = "%s_s2" % (efa)

    if pb == "T":
        tb_name = "%s_s0" % (efb)
    else:
        tb_name = "%s_s2" % (efb)

    if pb == "T":
        cl_type = "cl_" + map_types[pb] + map_types[pa]
    else:
        cl_type = "cl_" + map_types[pa] + map_types[pb]

    lbin = data[efa, efb]["lbin"]
    cb = data[efa, efb][pa + pb]

    spec_sacc.add_ell_cl(cl_type, ta_name, tb_name, lbin, cb)

In [ ]:
spec_sacc.save_fits("%s/data_sacc_smooth_%s.fits" % (namedir, sim_suffix), overwrite=True)

In [ ]:
sacc_data = sacc.Sacc.load_fits(
    packages_path + "/data/LensingLikelihood/lensing.sacc.fits"
)

ell, clkk, cov, ind = sacc_data.get_ell_cl(
    "cl_00", "ck", "ck", return_cov=True, return_ind=True
)
wins = sacc_data.get_bandpower_windows(ind)

print(sacc_data.metadata)
print(sacc_data.tracers)

In [ ]:
lensing = model_multi.components[0].likelihoods[1]

In [ ]:
s = sacc.Sacc()
s.metadata["info"] = "CMB lensing power spectra from reconstruction simulations"

beam = np.ones_like(ell)

s.add_tracer(
    tracer_type="Map",
    name="ck",
    quantity="cmb_convergence",
    spin=0,
    map_unit="uK_CMB",
    ell=ell,
    beam=beam,
)

s.add_ell_cl(
    "cl_00",  # Data type
    "ck",  # 1st tracer's name
    "ck",  # 2nd tracer's name
    ell,  # Effective multipole
    lensing._get_theory(**cosmo_params),  # Power spectrum values
    window=wins,  # Bandpower windows
)

cov = sacc.covariance.FullCovariance.from_hdu(cov)
s.add_covariance(cov, overwrite=False)

ndir = (
    packages_path
    + "/data/LensingLikelihood/soliket_lensing/smooth_data/"
    + "clkk_reconstruction_sim_frommulti_smooth.fits"
)

os.makedirs(
    packages_path + "/data/LensingLikelihood/soliket_lensing/smooth_data/",
    exist_ok=True,
)
s.save_fits(ndir, overwrite=True)

# Use with cross-covariance

In [ ]:
# Fiducial cosmo params. When used in a MCMC, the smooth data must agree
# with this input values

cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": 0.06,
    "nnu": 3.044,
}

common_nuis_params = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0,  # only ideal values for now
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

TT_nuis_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "xi": 0.20,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.0,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
}

TE_nuis_params = {
    "a_gte": 0.10,
    "a_pste": 0,
    "alpha_dE": -0.4,
}

EE_nuis_params = {
    "a_gee": 0.10,
    "a_psee": 0,
    "alpha_dE": -0.4,
    "calE_LAT_93": 1,
    "calE_LAT_145": 1,
    "calE_LAT_225": 1,
}

In [ ]:
namedir = packages_path + "/data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
mflike_options = {
    "data_folder": namedir,
    "input_file": pre + "smooth_00000.fits",
    "cov_Bbl_file": packages_path + "/data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
    "defaults": {
        "polarizations": ["TT", "TE", "ET", "EE"],
        "scales": {
            "TT": [30, 9000],
            "TE": [30, 9000],
            "ET": [30, 9000],
            "EE": [30, 9000],
        },
        "symmetrize": False,
    },
}

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/"
    "clkk_reconstruction_sim_frommulti_smooth.fits",
    "theory_lmax": 5000,
    "name": "kk",
    "stop_at_error": True,
}

info_multi_noxcov = {
    "likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {
        "camb": {
            "extra_args": {
                "bbn_predictor": "PArthENoPE_880.2_standard.dat",
                "WantTransfer": True,
                "Transfer.high_precision": True,
                "Transfer.kmax": 1.2,
                "halofit_version": "mead",
                "lens_potential_accuracy": 1,
                "num_massive_neutrinos": 1,
                "theta_H0_range": [20, 100],
            },
            "stop_at_error": True,
        }
    },
    "params": cosmo_params
    | common_nuis_params
    | TT_nuis_params
    | TE_nuis_params
    | EE_nuis_params,
    "packages_path": packages_path,
    "debug": False,
}
info_multi_noxcov["theory"]["mflike.BandpowerForeground"] = {"stop_at_error": True}

xcovpath = "../cross_cov/products/"

info_multi_xcov = {
    "likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "cross_cov_path": xcovpath + "XCov_full_sacc.fits",
            "stop_at_error": True,
        }
    },
    "theory": {
        "camb": {
            "extra_args": {
                "bbn_predictor": "PArthENoPE_880.2_standard.dat",
                "WantTransfer": True,
                "Transfer.high_precision": True,
                "Transfer.kmax": 1.2,
                "halofit_version": "mead",
                "lens_potential_accuracy": 1,
                "num_massive_neutrinos": 1,
                "theta_H0_range": [20, 100],
            },
            "stop_at_error": True,
        }
    },
    "params": cosmo_params
    | common_nuis_params
    | TT_nuis_params
    | TE_nuis_params
    | EE_nuis_params,
    "packages_path": packages_path,
    "debug": False,
}
info_multi_xcov["theory"]["mflike.BandpowerForeground"] = {"stop_at_error": True}

In [ ]:
from cobaya.model import get_model

model_multi_noxcov = get_model(info_multi_noxcov)

Here, you can see how to create a multi-Gaussian covariance that includes cross-covariance between mflike and lensing.

In [ ]:
import ast

from soliket.gaussian.gaussian_data import CrossCov

mflike_data = model_multi_noxcov.components[0].likelihoods[0]._get_gauss_data()
lensing_data = model_multi_noxcov.components[0].likelihoods[1]._get_gauss_data()

# Create CrossCov and add components with their full covariances
crosscov = CrossCov()

# Add mflike component
crosscov.add_component(name=mflike_data.name, cov=mflike_data.cov)

# Add lensing component
crosscov.add_component(name=lensing_data.name, cov=lensing_data.cov)

# Load old cross-covariance from NPZ
xcovpath = "../cross_cov/products/"
old_xcov_file = xcovpath + "XCov_full.npz"

# Use ast.literal_eval for safe parsing of tuple keys
old_cov = {ast.literal_eval(k): v for k, v in np.load(old_xcov_file).items()}
cross_cov_matrix = old_cov[("mflike", "kk")]

# Trim cross-covariance to match component shapes
cross_cov_trimmed = cross_cov_matrix[mflike_data.indices, :][:, lensing_data.indices]

# Add the cross-covariance between them
crosscov.add_cross_covariance(mflike_data.name, lensing_data.name, cross_cov_trimmed)

# Save to SACC format
crosscov.save(xcovpath + "XCov_full_sacc.fits")

In [ ]:
model_multi_xcov = get_model(info_multi_xcov)

In [ ]:
print(model_multi_noxcov.loglikes())
print(model_multi_xcov.loglikes())

In [ ]:
like = model_multi_xcov.likelihood["soliket.gaussian.MultiGaussianLikelihood"]
params = {
    **cosmo_params,
    **common_nuis_params,
    **TT_nuis_params,
    **EE_nuis_params,
    **TE_nuis_params,
}
theory = like._get_theory(**params)
delta = like.data.data.y - theory
like.data._fast_chi_squared(like.data.inv_cov, delta)

In [ ]:
like = model_multi_noxcov.likelihood["soliket.gaussian.MultiGaussianLikelihood"]
params = {
    **cosmo_params,
    **common_nuis_params,
    **TT_nuis_params,
    **EE_nuis_params,
    **TE_nuis_params,
}
theory = like._get_theory(**params)
delta = like.data.data.y - theory
like.data._fast_chi_squared(like.data.inv_cov, delta)